In [0]:
%pip install -U nbformat dspy mlflow --quiet
dbutils.library.restartPython()

In [0]:
import base64

import databricks.sdk
from databricks.sdk.service.workspace import ImportFormat
import nbformat
import numpy as np 

import pandas as pd
import dspy

In [0]:
import mlflow
import dspy 

mlflow.dspy.autolog()

In [0]:
lm = dspy.LM('databricks/databricks-dbrx-instruct')
dspy.settings.configure(lm=lm)

In [0]:
class SQLPrompt1(dspy.Signature):
    """ Check if the provided context has any code snippet which creates a table/view called workloads and get shcema of workloads """

    text: str = dspy.InputField()
    
    score: str = dspy.OutputField(desc="15 if any code snippet below creates a table/view and displays its schema, 7.5 if it creates a table/view but does not display its schema,  0 if no code snippet creates a table/view")
    
    code_snippet: str = dspy.OutputField(desc="provide the code snippet which creates a table/view and displays its schema encapsulated as a string") 

classify = dspy.ChainOfThought(SQLPrompt1)


In [0]:
train_df = spark.read.table("users.abhay_jalisatgi.training_set")
train_df = train_df.filter(" section = 'SQL' and trim(question) = 'Check if the provided context has any code snippet which creates a table/view called workloads and get shcema of workloads' ")
train_df.display()

In [0]:
training_examples = train_df.select("code_snippet", "score").collect()
trainset = [dspy.Example(text=x.code_snippet, score=x.score) for x in training_examples]
trainset[0].score

In [0]:
def validate_score(example, prediction, trace=None):
    return prediction.score == example.score

In [0]:
from dspy.teleprompt import * 

# Optimize
tp = dspy.MIPROv2(metric=validate_score
                  # , auto="light"
                  , num_candidates = 1
                  , num_threads=6
                  # , max_errors=0
                  # , verbose=True
                  )
optimized_classify = tp.compile(
                                classify
                                , trainset=trainset
                                , num_trials = 1
                                , minibatch_size = 3
                                , max_labeled_demos=0
                                , max_bootstrapped_demos=0
                                )

In [0]:
# Use lm.inspect_history(n=1) to get system_prompts 

In [0]:
from typing import Literal

class Score(dspy.Signature):
    """
        Assign score for a given context based on following instructions 

        1. Check if the provided context has any code snippet which creates a table/view called workloads and get shcema of workloads. 
           Assign a score of 15 if any code snippet below creates a table/view and displays its schema, 7.5 if it creates a table/view but does not display its schema,  0 if no code snippet creates a table/view

        2. 
    
    """

    event: str = dspy.InputField()
    category: Literal[
        "Wars and Conflicts",
        "Politics and Governance",
        "Science and Innovation",
        "Cultural and Artistic Movements",
        "Exploration and Discovery",
        "Economic Events",
        "Social Movements",
        "Man-Made Disasters and Accidents",
        "Natural Disasters and Climate",
        "Sports and Entertainment",
        "Famous Personalities and Achievements"
    ] = dspy.OutputField()
    confidence: float = dspy.OutputField()

classify = dspy.Predict(Categorize)

# Here is how we call this module
classification = classify(event="Second Boer War: In the Battle of Magersfontein the Boers commanded by general Piet Cronjé inflict a defeat on the forces of the British Empire commanded by Lord Methuen trying to relieve the Siege of Kimberley.")

In [0]:
classification

In [0]:
### Sample code for model switchin in context #### 

# with open("0101_events.json", 'r') as file:
#     data = json.load(file)
#     events = pd.DataFrame(data['events'])
    
#     # Using our small model
#     with dspy.context(lm=dspy.LM('ollama_chat/llama3.2:1b', api_base='http://localhost:11434')):
#         events['category_32_1b'], events['confidence_32_1b'] = zip(*events['description'].apply(classify_event))

#     # Using our large model
#     with dspy.context(lm=dspy.LM('ollama_chat/llama3.3', api_base='http://localhost:11434')):
#         events['category_33'], events['confidence_33'] = zip(*events['description'].apply(classify_event))

#     events.to_csv("model_compare.csv", index=False)

In [0]:
def validate_category(example, prediction, trace=None):
    return prediction.category == example.category

In [0]:
trainset = [] # Trainset is to be provided as a list of examples which sets up event and category 
evaluator = Evaluate(devset=trainset, num_threads=1, display_progress=True, display_table=5)
evaluator(classify, metric=validate_category)